<a href="https://colab.research.google.com/github/fabrytp8/AG1/blob/main/ej_video/parte_3_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os
df_train = pd.read_csv('partitions/train.csv',sep=';')
# df_test = pd.read_csv('partitions/test.csv',sep=';')
matriz_datos_train=feature_extraction(df_train)
matriz_datos_test =feature_extraction(df_test)
print(np.shape(matriz_datos_train))
print(np.shape(matriz_datos_test))
if not os.path.exists('features'):
    os.mkdir('features')
np.save('features/matriz_datos_train.npy',matriz_datos_train)
np.save('features/matriz_datos_test.npy',matriz_datos_test)


(77, 24)
(19, 24)


In [ ]:
def feature_extraction(df):
    import cv2
    import matplotlib.pyplot as plt
    fingerprint=[]
    for i in range(0,len(df)):
        # print('[info] --- Extrayendo información para la muestra',i)
        file = df.ID[i]
        img = cv2.imread('images/' + file)
        rnfl_mask = cv2.imread('rnfl/' + file,0)
        retina_mask = cv2.imread('retinas/' + file,0)
        # print(file)
        t, mask_rnfl = cv2.threshold(rnfl_mask,0,255,cv2.THRESH_OTSU) # 0 es por defecto y 1 es el valor máximo de la máscara
        t, mask_retina = cv2.threshold(retina_mask,0,255,cv2.THRESH_OTSU) # 0 es por defecto y 1 es el valor máximo de la máscara
            # print(mask_rnfl.shape[0])    
        for j in range(0,mask_rnfl.shape[1]):
            cv2.circle(mask_rnfl, (j,0), 0, (0,0,0), 0)
        for j in range(0,mask_rnfl.shape[0]):
            cv2.circle(mask_rnfl, (mask_rnfl.shape[1]-1,j), 0, (0,0,0), 0)
        for j in range(0,mask_rnfl.shape[0]):
            cv2.circle(mask_rnfl, (0,j), 0, (0,0,0), 0)
        for j in range(0,mask_rnfl.shape[1]):
            cv2.circle(mask_rnfl, (j,mask_rnfl.shape[0]-1), 0, (0,0,0), 0)

        for j in range(0,mask_retina.shape[1]):
            cv2.circle(mask_retina, (j,0), 0, (0,0,0), 0)
        for j in range(0,mask_retina.shape[0]):
            cv2.circle(mask_retina, (mask_retina.shape[1]-1,j), 0, (0,0,0), 0)
        for j in range(0,mask_retina.shape[0]):
            cv2.circle(mask_retina, (0,j), 0, (0,0,0), 0)
        for j in range(0,mask_retina.shape[1]):
            cv2.circle(mask_retina, (j,mask_retina.shape[0]-1), 0, (0,0,0), 0)
        #visualización
        # fig, ax =plt.subplots(1,3)
        # ax[0].imshow(img, cmap='gray'), ax[0].set_title('Imagen')
        # ax[1].imshow(mask_rnfl, cmap='gray'), ax[1].set_title('rnfl')
        # ax[2].imshow(mask_retina, cmap='gray'), ax[2].set_title('retina')

        #estadísticas unidimensionales
        thickness_rnfl=[]
        for j in range(0, mask_rnfl.shape[1]):
            pos = np.where(mask_rnfl[:,j]==255)
            if pos[0].size>0:
                thickness_rnfl.append(pos[0][-1]-pos[0][0])
        thickness_rnfl = np.array(thickness_rnfl)
        # print(thickness_rnfl)
        # caracteristicas de medida en tendencia central
        media = np.mean(thickness_rnfl)
        mediana = np.median(thickness_rnfl)

        #caracteristicas basadas en medidas de dispersión
        desvest = np.std(thickness_rnfl)
        #caractrísticas de distribución
        from scipy import stats
        asimetria = stats.skew(thickness_rnfl)
        curtosis = stats.kurtosis(thickness_rnfl)

        #otras características
        minimo = np.min(thickness_rnfl)
        maximo = np.max(thickness_rnfl)
        #fingerprin de la rnfl
        features_rnfl = [media,mediana,desvest, asimetria, curtosis,minimo,maximo]

        #carcaterisiticas bidimensionales
        from skimage.measure import regionprops
        prop = regionprops(mask_retina)
        bb = prop[0].bbox
        retina = img[bb[0]:bb[2], bb[1]:bb[3],0]
        # plt.imshow(retina,cmap='gray')
        # plt.show()

        #matriz de concurrencia GLCM
        from skimage.feature import graycomatrix, graycoprops
        GLCM = graycomatrix(retina, distances=[2], angles=[90],levels=256,symmetric=True, normed=True)
        contraste = graycoprops(GLCM, 'contrast')[0,0]
        disimilitud = graycoprops(GLCM, 'dissimilarity')[0,0]
        homogeneidad = graycoprops(GLCM, 'homogeneity')[0,0]
        ASM = graycoprops(GLCM, 'ASM')[0,0]
        energia = graycoprops(GLCM, 'energy')[0,0]
        correlacion = graycoprops(GLCM, 'correlation')[0,0]

        #LBP loca binary patterns
        from skimage.feature import local_binary_pattern
        R=1
        P=8*R
        lbp_image = local_binary_pattern(retina, P,R, method='uniform')
        lbp_image = np.uint8(lbp_image)
        hist_lbp = cv2.calcHist([lbp_image.ravel()],[0],None,[P+2],[0, P+2])
        his_lbp = hist_lbp.astype('float')
        hist_lbp /= (hist_lbp.sum() + 1e-7)
        hist_lbp= hist_lbp.tolist()
        hist_lbp = [item for sublist in hist_lbp for item in sublist]

        #visualización de la imagen LBP y el histograma
        # plt.imshow(retina,cmap='gray')
        # plt.show()
        # plt.imshow(lbp_image, cmap = 'gray')
        # plt.show()
        # plt.plot(hist_lbp)
        # plt.grid(True)
        # plt.show()
        #caracteristicas de textura(fingerprint retina)
        features_retina = [contraste,disimilitud, homogeneidad, ASM, energia, correlacion] + hist_lbp 

        #extraer informacion de la clase
        if df.Class[i]=='Healthy':
            etiqueta = [0]
        else:
            etiqueta = [1]
        fingerprint.append(features_rnfl + features_retina + etiqueta)
        # print(np.shape(fingerprint))
    matriz_datos = np.array(fingerprint)
    return matriz_datos